In [5]:
import os
import subprocess

# autoreload any .py scripts
%load_ext autoreload
%autoreload 2

# set the project's root directory as the notebooks' working directory
git_root = subprocess.run(['git', 'rev-parse', '--show-toplevel'], capture_output=True, text=True).stdout.strip()
os.chdir(git_root)
print(f"current working directory: {os.getcwd()}")

current working directory: /Users/nilsgandlau/code/browser-automation


In [2]:
# Imports
import json
import requests
from collections.abc import Callable
from typing import Annotated as A, Literal as L
from openai import OpenAI
from dotenv import load_dotenv
import os

from annotated_docs.json_schema import as_json_schema  # https://github.com/peterroelants/annotated-docs

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
client = OpenAI(api_key=openai_api_key)

In [21]:
from typing import Annotated, Literal

class StopException(Exception):
    """
    Stop Execution by raising this exception (Signal that the task is Finished).
    """

def multiply(a: int, b: int) -> str:
    return str(a*b)

def add(a: int, b: int) -> str:
    return str(a+b)

def finish(answer: Annotated[str, "Answer to the user's question."]) -> None:
    raise StopException(answer)

name_to_function_map: dict[str, Callable] = {
    "multiply": multiply, # idea: replace with `multiply.__name__: multiply` to make this even more programmatically
    "add": add,
    "finish": finish
}

function_schemas = [
    {"function": as_json_schema(func), "type": "function"}
    for func in name_to_function_map.values()
]

# Print the JSON Schemas
for schema in function_schemas:
    print(json.dumps(schema, indent=2))

{
  "function": {
    "name": "multiply",
    "description": "",
    "parameters": {
      "properties": {
        "a": {
          "type": "integer"
        },
        "b": {
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ],
      "type": "object"
    }
  },
  "type": "function"
}
{
  "function": {
    "name": "add",
    "description": "",
    "parameters": {
      "properties": {
        "a": {
          "type": "integer"
        },
        "b": {
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ],
      "type": "object"
    }
  },
  "type": "function"
}
{
  "function": {
    "name": "finish",
    "description": "",
    "parameters": {
      "properties": {
        "answer": {
          "type": "string"
        }
      },
      "required": [
        "answer"
      ],
      "type": "object"
    }
  },
  "type": "function"
}


In [59]:
messages = []
question_prompt = "Solve the following equation: 1+6*2"  # solution: 13
system_prompt = "Always respond using a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function). Reason step by step which actions to take to get to the answer. Only call functions with arguments coming verbatim from the user or the output of other functions."
#system_prompt = "Always start your response with: 'here we go:'. Before using a tool, you must provide your THOUGHTS or REASONING."
system_prompt = "Always respond using a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function)."

messages.append({"role": "system", "content": system_prompt})
messages.append({"role": "user", "content": question_prompt})

def run(messages: list[dict]) -> list[dict]:
    max_iterations = 5
    for i in range(max_iterations):
        print(f"--- Iteration {i=} ---")
        # send list of messages to get next response
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=function_schemas,
            tool_choice="auto",
        )
        response_message = response.choices[0].message
        print(response_message)
        messages.append(response_message)
        # check if GPT wanted to call a function
        tool_calls = response_message.tool_calls
        if tool_calls:
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                # Validate function name
                if function_name not in name_to_function_map:
                    print(f"Invalid function name: {function_name}")
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": f"Invalid function name: {function_name!r}"
                    })
                    continue
                # Get the function to call
                function_to_call: Callable = name_to_function_map[function_name]
                # Try getting the function arguments
                try:
                    function_args_dict = json.loads(tool_call.function.arguments)
                except json.JSONDecodeError as exc:
                    # JSON decoding failed
                    print(f"Error decoding function arguments: {exc}")
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name, 
                        "content": f"Error decoding function call `{function_name}` arguments {tool_call.function.arguments!r}! Error: {exc!s}",
                    })
                    continue
                # Call the selected function with generated arguments
                try:
                    print(f"Calling function {function_name} with args: {json.dumps(function_args_dict)}")
                    function_response = function_to_call(**function_args_dict)
                    # Extend conversation with function response
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    })
                except StopException as exc:
                    # Agent wants to stop the conversation (expected)
                    print(f"Finish task with message: '{exc!s}'")
                    return messages
                except Exception as exc:
                    # Unexpected error calling the function
                    print(f"Error calling function `{function_name}`: {type(exc).__name__}: {exc!s}")
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": f"Error calling function `{function_name}`: {type(exc).__name__}: {exc!s}",
                    })
                    continue
    return messages

In [60]:
messages = run(messages)

--- Iteration i=0 ---
ChatCompletionMessage(content='THOUGHT: According to the order of operations (PEMDAS/BODMAS), I need to perform the multiplication first and then the addition.\n\n1. Calculate \\(6 * 2\\)\n2. Add 1 to the result of step 1\n\nACTION: Call the multiply function to calculate \\(6 * 2\\)', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Rlnz3cAg4xGpHAydArKJ1jEV', function=Function(arguments='{"a":6,"b":2}', name='multiply'), type='function')])
Calling function multiply with args: {"a": 6, "b": 2}
--- Iteration i=1 ---
ChatCompletionMessage(content='THOUGHT: Now that I have the multiplication result (12), I need to add 1 to it.\n\nACTION: Call the add function to calculate \\(1 + 12\\)', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gZsDgrPOaM4ICSQsmAF1NGvW', function=Function(arguments='{"a":1,"b":12}', name='add'), type='function')])
Calling function add with args: {"a": 1, "b": 12}
-

In [61]:
print(len(messages))

7


In [62]:
messages

[{'role': 'system',
  'content': 'Always respond using a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function).'},
 {'role': 'user', 'content': 'Solve the following equation: 1+6*2'},
 ChatCompletionMessage(content='THOUGHT: According to the order of operations (PEMDAS/BODMAS), I need to perform the multiplication first and then the addition.\n\n1. Calculate \\(6 * 2\\)\n2. Add 1 to the result of step 1\n\nACTION: Call the multiply function to calculate \\(6 * 2\\)', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Rlnz3cAg4xGpHAydArKJ1jEV', function=Function(arguments='{"a":6,"b":2}', name='multiply'), type='function')]),
 {'tool_call_id': 'call_Rlnz3cAg4xGpHAydArKJ1jEV',
  'role': 'tool',
  'name': 'multiply',
  'content': '12'},
 ChatCompletionMessage(content='THOUGHT: Now that I have the multiplication result (12), I

In [58]:
from termcolor import colored  

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if isinstance(message, dict):
            if message["role"] == "system":
                print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
            elif message["role"] == "user":
                print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
            elif message["role"] == "assistant" and message.get("function_call"):
                print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
            elif message["role"] == "assistant" and not message.get("function_call"):
                print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
            elif message["role"] == "function":
                print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))
        else:
            # message is a function/tool call!
            print(message)

pretty_print_conversation(messages)

# for message in messages:
#     if not isinstance(message, dict):
#         message = message.model_dump()  # Pydantic model
#     print(json.dumps(message, indent=2))

system: Always respond using a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function). Reason step by step which actions to take to get to the answer. Only call functions with arguments coming verbatim from the user or the output of other functions.

user: Solve the following equation: 1+6*2

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_MxH5FQuU75z9kkWSgxiT19lU', function=Function(arguments='{"a":6,"b":2}', name='multiply'), type='function')])
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wzHmiwj8iN0l4zZuHTIlNWbJ', function=Function(arguments='{"a":1,"b":12}', name='add'), type='function')])
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall

## OpenAI ChatCompletion with tools + images

In [67]:
import requests
import json

api_key = openai_api_key
endpoint = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

data = {
    "model": "gpt-4o",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant. You have access to the following tools: multiply, add."},
        {"role": "user", "content": "What is 1+6*2?"},
    ],
    "functions": [
        {
            "name": "multiply",
            "description": "Use this to multiply two numbers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number to be multiplied."},
                    "b": {"type": "number", "description": "Second number to be multiplied"}
                },
                "required": ["a", "b"]
            }
        },
        {
            "name": "add",
            "description": "Use this to add two numbers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "First number to be added."},
                    "b": {"type": "number", "description": "Second number to be added"}
                },
                "required": ["a", "b"]
            }
        }
    ]
}
response = requests.post(endpoint, headers=headers, data=json.dumps(data))
print(response.json())


{'id': 'chatcmpl-9bTXZuw0tzIfHt1VEqiJ4jN2FQsbV', 'object': 'chat.completion', 'created': 1718718717, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "To evaluate the expression \\(1 + 6 \\times 2\\), we need to follow the order of operations (PEMDAS/BODMAS):\n\n1. First, perform the multiplication:\n\\[ 6 \\times 2 = 12 \\]\n\n2. Then, perform the addition:\n\\[ 1 + 12 \\]\n\nI'll calculate the final addition.", 'function_call': {'name': 'add', 'arguments': '{\n  "a": 1,\n  "b": 12\n}'}}, 'logprobs': None, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 124, 'completion_tokens': 95, 'total_tokens': 219}, 'system_fingerprint': 'fp_f4e629d0a5'}


In [77]:
result = response.json()
print(result["choices"][0]["message"]["content"])
result

To evaluate the expression \(1 + 6 \times 2\), we need to follow the order of operations (PEMDAS/BODMAS):

1. First, perform the multiplication:
\[ 6 \times 2 = 12 \]

2. Then, perform the addition:
\[ 1 + 12 \]

I'll calculate the final addition.


{'id': 'chatcmpl-9bTXZuw0tzIfHt1VEqiJ4jN2FQsbV',
 'object': 'chat.completion',
 'created': 1718718717,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "To evaluate the expression \\(1 + 6 \\times 2\\), we need to follow the order of operations (PEMDAS/BODMAS):\n\n1. First, perform the multiplication:\n\\[ 6 \\times 2 = 12 \\]\n\n2. Then, perform the addition:\n\\[ 1 + 12 \\]\n\nI'll calculate the final addition.",
    'function_call': {'name': 'add',
     'arguments': '{\n  "a": 1,\n  "b": 12\n}'}},
   'logprobs': None,
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 124, 'completion_tokens': 95, 'total_tokens': 219},
 'system_fingerprint': 'fp_f4e629d0a5'}

## OpenAI Assistant API with Images

In [ ]:
# Q: Can I get OpenAI Assistant API to call a function 
# `extract_data_from_image()` after it sees an image?

In [7]:
# upload image to OpenAI
file = client.files.create(
    file=open("screenshots/79.jpg", "rb"),
    purpose="vision"
)

In [39]:
assistant = client.beta.assistants.create(
  name="Image Describer",
  description="You are great at describing images.",
  model="gpt-4o",
)

In [40]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Describe what you see in the image.",
            "attachments": [
                {
                    "file_id": file.id,
                    "tools": [{"type": "code_interpreter"}]
                }
            ]
        }
    ]
)
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
# wait ... status can be 'queued'!
messages = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
print(thread)
print(run)
print(messages)

Thread(id='thread_Aa1ggTLpJrHbWSCNbnLeEkLM', created_at=1718724836, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-7BHa88wkH1ikNWOOM7TJIxKt']), file_search=None))
Run(id='run_9wbRQZzqE2BpB1xyvNTGNjmD', assistant_id='asst_qlDE9IWhi6tuDzdVQWUnHCLM', cancelled_at=None, completed_at=None, created_at=1718724837, expires_at=1718725437, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Aa1ggTLpJrHbWSCNbnLeEkLM', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})
SyncCursorPage[Message](data=[], object='list', first_id=None, last_id=None, has_more=False)


In [43]:
messages = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
print(f"Annotations: {messages.data[0].content[0].text.annotations}")
print(f"Message: {messages.data[0].content[0].text.value}")

Annotations: []
Message: I cannot view the image you uploaded due to current limitations. Could you please provide a description or any details about the image? This would help me assist you better.


In [46]:
# alternative
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Describe what you see in the image."
        },
        {
          "type": "image_file",
          "image_file": {"file_id": file.id}
        },
      ],
    }
  ]
)
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
# wait ... status can be 'queued'!
messages = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
print(thread)
print(run)
print(messages)

Thread(id='thread_rcH2oiwkEGHq3fKEVLc63b2t', created_at=1718724934, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Run(id='run_Spyev72UCh7D6SYv9QG5qi3w', assistant_id='asst_qlDE9IWhi6tuDzdVQWUnHCLM', cancelled_at=None, completed_at=None, created_at=1718724934, expires_at=1718725534, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_rcH2oiwkEGHq3fKEVLc63b2t', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})
SyncCursorPage[Message](data=[], object='list', first_id=None, last_id=None, has_more=False)


In [47]:
messages = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
print(f"Annotations: {messages.data[0].content[0].text.annotations}")
print(f"Message: {messages.data[0].content[0].text.value}")

Annotations: []
Message: The image depicts a booking schedule for tennis courts, specifically for Friday, June 14, 2024. The schedule is laid out in a grid format, with time slots listed vertically along the left side, starting at 10:00 and ending at 21:30 in 30-minute increments. Horizontally along the top, there are labels for different courts or places (Platz 1 through Platz 10 and Ballwand Zeit).

Each cell in the grid represents a 30-minute time slot for a specific court. The cells are color-coded:
- Red cells are likely indicating that the time slot is already booked.
- Blue cells with the word "BUCHEN" (German for "book") indicate that these time slots are available for booking.

The page header includes options such as "Startseite" (home page), "Camps," "Events & Kurse" (events and courses), "Tennis Halle" (tennis hall), "Guthaben & Gutscheine" (credits and vouchers), "Login," and "Registrieren" (register).

At the bottom, there are various icons and links including "Open in ei

### Assistant function calling

In [105]:
get_current_temperature_json = {
    "type": "function",
    "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
                "type": "string",
                "enum": ["Celsius", "Fahrenheit"],
                "description": "The temperature unit to use. Infer this from the user's location."
            }
            },
            "required": ["location", "unit"]
        }
    }
}

get_rain_probability_json = {
    "type": "function",
    "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
            }
            },
            "required": ["location"]
        }
    }
}

assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use provided functions to answer questions.",
    model="gpt-4o",
    tools=[get_current_temperature_json, get_rain_probability_json]
)
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what is the weather in frankfurt, germany?"
)

#! This does not work:
# run = client.beta.threads.runs.create(
    # thread_id=thread.id,
    # assistant_id=assistant.id
# )

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait ... status can be 'queued'!
messages = client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id)
print(thread)
print(run)
print(messages)

Thread(id='thread_lOXX3gklVAHcxdrLdtruY5oI', created_at=1718726422, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Run(id='run_NIkIyWEGxmrpauHQg1r38QCM', assistant_id='asst_H6b2d5WKJQBJAYRKKKycTzgt', cancelled_at=None, completed_at=None, created_at=1718726423, expires_at=1718727023, failed_at=None, incomplete_details=None, instructions='You are a weather bot. Use provided functions to answer questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_xOrsZWPXjl9qSfoJojRGjN7b', function=Function(arguments='{"location": "Frankfurt, Germany", "unit": "Celsius"}', name='get_current_temperature'), type='function'), RequiredActionFunctionToolCall(id='call_K8ndyNHA0rp0SKcBIOm3h8Nm', function=Function(arguments=

In [106]:
# if a function is called, the status of the `run` changes to `"requires_action"`
run.status

'requires_action'

In [107]:
# In that case, GPT wants to call a certain function with certain arguments.
print(run.required_action)
print()
print(run.required_action.submit_tool_outputs.tool_calls)
print()

for tool in run.required_action.submit_tool_outputs.tool_calls:
    print("GPT wants to call the following function: ")
    print(f"\t* tool name: {tool.function.name}")
    print(f"\t* tool args: {tool.function.arguments}")

RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_xOrsZWPXjl9qSfoJojRGjN7b', function=Function(arguments='{"location": "Frankfurt, Germany", "unit": "Celsius"}', name='get_current_temperature'), type='function'), RequiredActionFunctionToolCall(id='call_K8ndyNHA0rp0SKcBIOm3h8Nm', function=Function(arguments='{"location": "Frankfurt, Germany"}', name='get_rain_probability'), type='function')]), type='submit_tool_outputs')

[RequiredActionFunctionToolCall(id='call_xOrsZWPXjl9qSfoJojRGjN7b', function=Function(arguments='{"location": "Frankfurt, Germany", "unit": "Celsius"}', name='get_current_temperature'), type='function'), RequiredActionFunctionToolCall(id='call_K8ndyNHA0rp0SKcBIOm3h8Nm', function=Function(arguments='{"location": "Frankfurt, Germany"}', name='get_rain_probability'), type='function')]

GPT wants to call the following function: 
	* tool name: get_current_temperature
	* tool args: {"location": "Frankfurt, 

In [108]:
# simulate function responses:
tool_outputs = []
for tool in run.required_action.submit_tool_outputs.tool_calls:
    if tool.function.name == "get_current_temperature":
        tool_outputs.append({"tool_call_id": tool.id, "output": "30°C"})
    if tool.function.name == "get_rain_probability":
        tool_outputs.append({"tool_call_id": tool.id, "output": "80%"})

# submit all tool outputs
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")


Tool outputs submitted successfully.


In [118]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
  print()
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_x1Re2LPGRtT1XOk2G7s7rNHu', assistant_id='asst_H6b2d5WKJQBJAYRKKKycTzgt', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The current temperature in Frankfurt, Germany is 30°C, with a 80% probability of rain.'), type='text')], created_at=1718726432, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_NIkIyWEGxmrpauHQg1r38QCM', status=None, thread_id='thread_lOXX3gklVAHcxdrLdtruY5oI'), Message(id='msg_AysOabRRcvPwTXahQzr5PyBB', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='what is the weather in frankfurt, germany?'), type='text')], created_at=1718726422, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_lOXX3gklVAHcxdrLdtruY5oI')], object='list', first_id='msg_x1Re2LPGRtT1XOk2G7s7rN

### make the agent more verbose

In [179]:
assistant = client.beta.assistants.create(
    instructions="Always respond using a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function).",
    # instructions="You are a researcher that thinks step-by-step before every response. If you use a function, explain your reasoning. You must repeat back the function output verbatim when answering, without alterations.",
    model="gpt-4o",
    tools=[get_current_temperature_json, get_rain_probability_json]
)
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what is the weather in frankfurt, germany?"
)
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [180]:
# simulate function responses:
tool_outputs = []
for tool in run.required_action.submit_tool_outputs.tool_calls:
    if tool.function.name == "get_current_temperature":
        tool_outputs.append({"tool_call_id": tool.id, "output": "couldnt get the temperature!"})
    if tool.function.name == "get_rain_probability":
        tool_outputs.append({"tool_call_id": tool.id, "output": "80%"})

# submit all tool outputs
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")

Tool outputs submitted successfully.


In [182]:
from termcolor import colored


if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
  print()
  print(colored(messages.data[1].content[0].text.value, color="green"))
  print(colored(messages.data[0].content[0].text.value, color="red"))
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_Vtit6kUQJ9BAxMqcHhIuQLQz', assistant_id='asst_XZckIsxT3vcUlp2rlk0BHEGq', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="It looks like I couldn't retrieve the current temperature for Frankfurt, Germany. However, the probability of rain there is 80%."), type='text')], created_at=1718742908, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_pIQ8UUeSmoCu4o6USLAExNu5', status=None, thread_id='thread_INYhVPRoWaaVdL2GWd6djvTl'), Message(id='msg_bXju9JRwoTPk9RYV6qhRZKQ7', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='what is the weather in frankfurt, germany?'), type='text')], created_at=1718742901, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_INYhVPRoWaaVdL2GWd6djvTl')], object='l

In [154]:
print(run.instructions)
print(run.tools)
print(run.tool_choice)

You are a researcher that thinks step-by-step before every response. If you use a function, explain your reasoning. Describe the output of the function calls using 'OBSERVATION:' prefix.
[FunctionTool(function=FunctionDefinition(name='get_current_temperature', description='Get the current temperature for a specific location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g., San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['Celsius', 'Fahrenheit'], 'description': "The temperature unit to use. Infer this from the user's location."}}, 'required': ['location', 'unit']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_rain_probability', description='Get the probability of rain for a specific location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g., San Francisco, CA'}}, 'required': ['location']}), type='function')]
auto


In [163]:
for tool in run.tools:
    print(tool.function.name)
    print(tool.function.parameters)

get_current_temperature
{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g., San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['Celsius', 'Fahrenheit'], 'description': "The temperature unit to use. Infer this from the user's location."}}, 'required': ['location', 'unit']}
get_rain_probability
{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g., San Francisco, CA'}}, 'required': ['location']}


In [164]:
run.tool_resources

{}

In [166]:
messages

SyncCursorPage[Message](data=[Message(id='msg_OrF4TeGGO4JBXmkCqrPgwpRr', assistant_id='asst_6Y3v0Xgc6ztOPc7UBjulADTI', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='OBSERVATION: \n1. The current temperature in Frankfurt, Germany is 30°C.\n2. The probability of rain in Frankfurt, Germany is 80%.\n\nSUMMARY: \nThe temperature in Frankfurt is currently 30°C, and there is an 80% chance of rain.'), type='text')], created_at=1718738598, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_YysAhwZQGeDIgdfeNWKsoDkT', status=None, thread_id='thread_0PBLyh5iCfL00tJC7aQklhvK'), Message(id='msg_YGXHKP4eP1eANb1DmxAWBOJ0', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='what is the weather in frankfurt, germany?'), type='text')], created_at=1718738589, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.messa